In [1]:
import os
import sys
from oligo_designer_toolsuite.oligo_property_filter import (
    PropertyFilter,
    GCContent,
    MeltingTemperature,
    ConsecutiveRepeats,
    Secondary_struct
)
import pandas as pd

from oligo_designer_toolsuite.experiment_specific import(
PrimerProbes
)

#sys.path.append(os.path.dirname(os.getcwd()))
ROOT_DIR = "/home/cw/PycharmProjects/oligo-designer-toolsuite_merfish"
os.chdir(ROOT_DIR)
import yaml
from Bio.SeqUtils import MeltingTemp as mt
os.getcwd()
config_path = "/home/cw/PycharmProjects/oligo-designer-toolsuite_merfish/tutorials/configs/merfish_probe_designer_test.yaml"
test_primer_probs = PrimerProbes(30,30,config_path)

In [2]:
test_primer_probs.create_primer1()

AttributeError: 'dict' object has no attribute 'oligos_DB'

In [3]:
test_primer_probs = PrimerProbes(30,30,"/home/cw/PycharmProjects/oligo-designer-toolsuite_merfish/tutorials/configs/merfish_probe_designer_test.yaml")
#PrimerProbes.create_primer1()

TypeError: MeltingTemperature.__init__() got an unexpected keyword argument 'arm_Tm_max'

In [5]:
config_file = os.path.join(ROOT_DIR,"tutorials","configs","merfish_probe_designer_test.yaml")

with open(config_file, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
dir_output = os.path.join(os.getcwd(),"test_merfish", config["dir_output"]) # create the complete path for the output directory
dir_output

'/home/cw/PycharmProjects/oligo-designer-toolsuite_merfish/test_merfish/output'

In [12]:

oligo_database = pd.read_csv('output/output/oligos_creation/oligo_DB_human_GRCh38_NCBI_release_110_transcripts.tsv',sep='\t')
oligo_database.head()

,id,sequence,gene,transcript_id,exon_id,chromosome,start,end,strand,length,melting_temperature,GC_content
0,CYBA_1,ACCCAGTGGTACTTTGGTGCCTACTC,CYBA,NM_000101.4:XM_011522905.4;NM_000101.4:XM_0115...,NM_000101.4_exon3_NM_000101.4_exon2:XM_0115229...,16,88647146;88648085,88647172;88648111,-,26,59.897116,53.85
1,CYBA_2,CCCAGTGGTACTTTGGTGCCTACTCC,CYBA,NM_000101.4:XM_011522905.4;NM_000101.4:XM_0115...,NM_000101.4_exon3_NM_000101.4_exon2:XM_0115229...,16,88647147;88648086,88647173;88648112,-,26,60.446422,57.69
2,CYBA_3,CCAGTGGTACTTTGGTGCCTACTCCA,CYBA,NM_000101.4:XM_011522905.4;NM_000101.4:XM_0115...,NM_000101.4_exon3_NM_000101.4_exon2:XM_0115229...,16,88647148;88648087,88647174;88648113,-,26,59.897258,53.85
3,CYBA_4,CAGTGGTACTTTGGTGCCTACTCCAT,CYBA,NM_000101.4:XM_011522905.4;NM_000101.4:XM_0115...,NM_000101.4_exon3_NM_000101.4_exon2:XM_0115229...,16,88647149;88648088,88647175;88648114,-,26,58.301068,50.00
4,CYBA_5,AGTGGTACTTTGGTGCCTACTCCATT,CYBA,NM_000101.4:XM_011522905.4,NM_000101.4_exon3_NM_000101.4_exon2:XM_0115229...,16,88647150,88647176,-,26,57.720478,46.15


In [7]:
# the melting temperature params need to be preprocessed
Tm_params = config["Tm_parameters"]["shared"].copy()
Tm_params.update(config["Tm_parameters"]["property_filter"])
Tm_params["nn_table"] = getattr(mt, Tm_params["nn_table"])
Tm_params["tmm_table"] = getattr(mt, Tm_params["tmm_table"])
Tm_params["imm_table"] = getattr(mt, Tm_params["imm_table"])
Tm_params["de_table"] = getattr(mt, Tm_params["de_table"])

Tm_params = config["Tm_parameters"]["shared"].copy()
Tm_correction_param = config["Tm_correction_parameters"]["shared"].copy()
Tm_correction_param.update(config["Tm_correction_parameters"]["property_filter"])

In [8]:
#property filter for primer
primers_param = config["primers_setup"].copy()
primers_param

{'GC_content_min': 40,
 'GC_content_max': 60,
 'GC_clamp_n': 2,
 'Tm_min': 70,
 'Tm_max': 80,
 'Repeat_AA_max': 2}

In [9]:
# initialize the property filters for primer

gc_content_primer = GCContent(GC_content_min=primers_param["GC_content_min"], GC_content_max=primers_param["GC_content_max"])
melting_temperature_primer = MeltingTemperature(
    Tm_min=primers_param["Tm_min"],
    Tm_max=primers_param["Tm_max"],
    Tm_parameters=Tm_params,
    Tm_correction_parameters=Tm_correction_param
)
consecutive_repeats = ConsecutiveRepeats(primers_param["Repeat_AA_max"])
secondary_structure = Secondary_struct()

In [ ]:

# create the list of filters
filters = [gc_content, melting_temperature]

# initialize the property filter class
property_filter = PropertyFilter(filters=filters)

# filter the database
oligo_database = property_filter.apply(oligo_database=oligo_database, n_jobs=config["n_jobs"])
# write the intermediate result in a file
if config["write_intermediate_steps"]:
    oligo_database.write_oligos_DB(format=config["file_format"], dir_oligos_DB="property_filter")

In [ ]:
from oligo_designer_toolsuite.database import CustomReferenceDB, NcbiReferenceDB, EnsemblReferenceDB
from oligo_designer_toolsuite.oligo_specificity_filter import (
    SpecificityFilter,
    ExactMatches,
    LigationRegionCreation,
    BowtieSeedRegion,
    Blastn,
)

dir_specificity = os.path.join(dir_output, "specificity_temporary") # folder where the temporary files will be written

# generate the reference
reference_database = CustomReferenceDB(
    species=oligo_database.species,
    genome_assembly=oligo_database.genome_assembly,
    annotation_release=oligo_database.annotation_release,
    files_source=oligo_database.files_source,
    annotation_file=oligo_database.annotation_file,
    sequence_file=oligo_database.sequence_file,
    dir_output=dir_output
)
reference_database.create_reference_DB(block_size=config["block_size"]) # leave the standard parameters

# intialize the filter classes
exact_mathces = ExactMatches(dir_specificity=dir_specificity)
seed_ligation = LigationRegionCreation(ligation_region_size=config["ligation_region_size"])
seed_region = BowtieSeedRegion(dir_specificity=dir_specificity, seed_region_creation=seed_ligation)
blastn = Blastn(
    dir_specificity=dir_specificity,
    word_size=config["word_size"],
    percent_identity=config["percent_identity"],
    coverage=config["coverage"],
)
filters = [exact_mathces, seed_region, blastn]

# initialize the specificity filter class
specificity_filter = SpecificityFilter(filters=filters, write_genes_with_insufficient_oligos=config["write_removed_genes"])
# filte r the database
oligo_database = specificity_filter.apply(oligo_database=oligo_database, reference_database=reference_database, n_jobs=config["n_jobs"])
# write the intermediate result
if config["write_intermediate_steps"]:
    oligo_database.write_oligos_DB(format=config["file_format"], dir_oligos_DB="specificity_filter")